## Day 18: Duet

You discover a tablet containing some strange assembly code labeled simply "Duet". Rather than bother the sound card with it, you decide to run the code yourself. Unfortunately, you don't see any documentation, so you're left to figure out what the instructions mean on your own.

### Part One

It seems like the assembly is meant to operate on a set of registers that are each named with a single letter and that can each hold a single integer. You suppose each register should start with a value of 0.

In [375]:
import re
from collections import defaultdict

# Just to simplify: each register defaults to zero and can be read as attributes

class Registers(defaultdict):
    pat_register = re.compile('[a-z]')
    
    def __getattr__(self, k):
        if Registers.pat_register.match(k):
            return self[k]
        raise AttributeError('Register {} not found'.format(k))
        
# tests
registers = Registers(int)
registers['a'] = 23
assert registers.a == 23
print('ok')

ok


There aren't that many instructions, so it shouldn't be hard to figure out what they do. Here's what you determine:

 - `snd X` plays a sound with a frequency equal to the value of X.
 
 - `set X Y` sets register X to the value of Y.
 
 - `add X Y` increases register X by the value of Y.
 
 - `mul X Y` sets register X to the result of multiplying the value contained in register X by the value of Y.
 
 - `mod X Y` sets register X to the remainder of dividing the value contained in register X by the value of Y (that is, it sets X to the result of X modulo Y).
 
 - `rcv X` recovers the frequency of the last sound played, but only when the value of X is not zero. (If it is zero, 
 the command does nothing.)
 
 - `jgz X Y` jumps with an offset of the value of Y, but only if the value of X is greater than zero. (An offset of 2 skips the next instruction, an offset of -1 jumps to the previous instruction, and so on.)

Many of the instructions can take either a register (a single letter) or a number. The value of a register is the integer it contains; the value of a number is that number.

After each jump instruction, the program continues with the instruction to which the jump jumped. After any other instruction, the program continues with the next instruction. Continuing (or jumping) off either end of the program terminates it.

> let's do somethig minimal

In [376]:
class Proc:
    
    def __init__(self):
        self.registers = Registers(int)
        self.pc = 0
        self.code = []
        self.kernel = {
            'set': self.op_set
            }
        
    def op_set(self, register, value):
        self.registers[register] = int(value)
        
    def exec(self, line):
        cod_op, *args = line.split(' ')
        method = self.kernel[cod_op]
        method(*args)
        
p = Proc()
assert p.registers.a == 0
p.exec('set a 23')
assert p.registers.a == 23
print('ok')

ok


> Problem is that value in operation can be a constant (number) or another register. Les fix that.

In [377]:
class Proc:
    
    def __init__(self):
        self.registers = Registers(int)
        self.pc = 0
        self.code = []
        self.kernel = {
            'set': self.op_set
            }
        
    def get_value(self, s):
        return self.registers[s] if self.registers.pat_register.match(s) else int(s)
        
    def op_set(self, register, value):
        self.registers[register] = self.get_value(value)
        
    def exec(self, line):
        cod_op, *args = line.split(' ')
        method = self.kernel[cod_op]
        method(*args)
        
p = Proc()
p.exec('set b 11')
p.exec('set a b')
assert p.registers.b == 11
assert p.registers.a == 11
print('ok')

ok


> Ok, let's go with the rest of operations.

In [421]:
class Proc:
    
    def __init__(self):
        self.registers = Registers(int)
        self.pc = 0
        self.code = []
        self._clock = 0
        self.freq = None
        self.end_flag = False
        self.kernel = {
            'set': self.op_set,
            'snd': self.op_snd,
            'add': self.op_add,
            'mul': self.op_mul,
            'mod': self.op_mod,
            'rcv': self.op_rcv,
            'jgz': self.op_jgz,
            }
        self.stats = defaultdict(int)
    
    def get_value(self, s):
        return self.registers[s] if self.registers.pat_register.match(s) else int(s)

    def op_snd(self, register_or_value):
        self.freq = self.get_value(register_or_value)
        print('<<Play note on freq {}>>'.format(self.freq), end=' ')
        
    def op_set(self, register, value):
        self.registers[register] = self.get_value(value)
        
    def op_add(self, register, value):
        self.registers[register] += self.get_value(value)
        
    def op_mul(self, register, value):
        self.registers[register] *= self.get_value(value)

    def op_mod(self, register, value):
        self.registers[register] = self.registers[register] % self.get_value(value)
        
    def op_rcv(self, register):
        if self.registers[register] != 0:
            print('<<recover freq: {}>>'.format(self.freq), end=' ')
            self.end_flag = True
                
    def op_jgz(self, register, value):
        flag = self.get_value(register)
        if flag > 0:
            self.pc += self.get_value(value) - 1
                
    def exec(self, line):
        cod_op, *args = line.split(' ')
        self.stats[cod_op] = self.stats.setdefault(cod_op, 0) + 1
        method = self.kernel[cod_op]
        return method(*args)
        
    def must_exit(self):
        return any([
            self.pc >= len(self.code),
            self.end_flag,
        ])
    
    def run(self, lines, tron=False):
        self.code = [_ for _ in lines]
        self.pc = 0
        while True:
            line = self.code[self.pc]
            if tron:
                print('{:>8} PC: {} | {} [{}]'.format(
                    self._clock,
                    self.pc,
                    self.code[self.pc],
                    self.dump()
                    ),
                    end=' -> '
                )
            self.exec(line)
            self.pc += 1
            self._clock += 1
            if tron:
                print(self.dump())
            if self.must_exit():
                break
        print('{:>8} PC: {} | end execution'.format(
            self._clock,
            self.pc,
        ))
        
    def dump(self):
        buff = [
            '{}={}'.format(k, self.registers[k])
            for k in self.registers
            ]
        return ':'.join(buff)
        
p = Proc()
p.exec('set b 11')
p.exec('set a 3')
p.exec('add a b')
assert p.registers.b == 11
assert p.registers.a == 14
p.exec('add a -2')
assert p.registers.a == 12
print('ok')

ok


For example:

    set a 1
    add a 2
    mul a a
    mod a 5
    snd a
    set a 0
    rcv a
    jgz a -1
    set a 1
    jgz a -2

 - The first four instructions set a to 1, add 2 to it, square it, and then set it to itself modulo 5, resulting in a value of 4.
 
 - Then, a sound with frequency 4 (the value of a) is played.
 
 - After that, a is set to 0, causing the subsequent rcv and jgz instructions to both be skipped (rcv because a is 0, and jgz because a is not greater than 0).
 
- Finally, a is set to 1, causing the next jgz instruction to activate, jumping back two instructions to another jump, which jumps again to the rcv, which ultimately triggers the recover operation.

At the time the recover operation is executed, the frequency of the last sound played is 4.

> It works with the example?

In [422]:
p = Proc()
p.run([
    'set a 1',
    'add a 2',
    'mul a a',
    'mod a 5',
    'snd a',
    'set a 0',
    'rcv a',
    'jgz a -1',
    'set a 1',
    'jgz a -2',
], tron=True)
assert p.freq == 4
assert p.stats['set'] == 3
print('ok')

       0 PC: 0 | set a 1 [] -> a=1
       1 PC: 1 | add a 2 [a=1] -> a=3
       2 PC: 2 | mul a a [a=3] -> a=9
       3 PC: 3 | mod a 5 [a=9] -> a=4
       4 PC: 4 | snd a [a=4] -> <<Play note on freq 4>> a=4
       5 PC: 5 | set a 0 [a=4] -> a=0
       6 PC: 6 | rcv a [a=0] -> a=0
       7 PC: 7 | jgz a -1 [a=0] -> a=0
       8 PC: 8 | set a 1 [a=0] -> a=1
       9 PC: 9 | jgz a -2 [a=1] -> a=1
      10 PC: 7 | jgz a -1 [a=1] -> a=1
      11 PC: 6 | rcv a [a=1] -> <<recover freq: 4>> a=1
      12 PC: 7 | end execution
ok


> It works with the example

What is the value of the recovered frequency (the value of the most recently played sound) the first time a rcv instruction is executed with a non-zero value?

> Load the code

In [423]:
with open('input.txt', 'r') as f:
    program = [_.strip() for _ in f.readlines() if _.strip()]

for line in program[0:5]:
    print(line)

set i 31
set a 1
mul p 17
jgz p p
mul a 2


In [424]:
p = Proc()
p.run(program, tron=False)
print()
print('Part one:', p.freq)

<<Play note on freq 7656>> <<Play note on freq 8238>> <<Play note on freq 6004>> <<Play note on freq 8332>> <<Play note on freq 1278>> <<Play note on freq 4390>> <<Play note on freq 2853>> <<Play note on freq 5192>> <<Play note on freq 3260>> <<Play note on freq 2145>> <<Play note on freq 619>> <<Play note on freq 2038>> <<Play note on freq 577>> <<Play note on freq 5482>> <<Play note on freq 6474>> <<Play note on freq 2518>> <<Play note on freq 6607>> <<Play note on freq 4679>> <<Play note on freq 9367>> <<Play note on freq 6571>> <<Play note on freq 7217>> <<Play note on freq 2333>> <<Play note on freq 5752>> <<Play note on freq 7305>> <<Play note on freq 7346>> <<Play note on freq 803>> <<Play note on freq 2055>> <<Play note on freq 3604>> <<Play note on freq 8625>> <<Play note on freq 1120>> <<Play note on freq 8340>> <<Play note on freq 7345>> <<Play note on freq 693>> <<Play note on freq 9212>> <<Play note on freq 1898>> <<Play note on freq 9469>> <<Play note on freq 2218>> <<Pla

### Part Two

As you congratulate yourself for a job well done, you notice that the documentation has been on the back of the tablet this entire time. While you actually got most of the instructions correct, there are a few key differences. This assembly code isn't about sound at all - it's meant to be run twice at the same time.

Each running copy of the program has its own set of registers and follows the code independently - in fact, the programs don't even necessarily run at the same speed. To coordinate, they use the send (snd) and receive (rcv) instructions:

 - `snd X` sends the value of X to the other program. These values wait in a queue until that program is ready to receive them. Each program has its own message queue, so a program can never receive a message it sent.

 - `rcv X` receives the next value and stores it in register X. If no values are in the queue, the program waits for a value to be sent to it. Programs do not continue to the next instruction until they have received a value. Values are received in the order they are sent.

Each program also has its own program ID (one 0 and the other 1); the register p should begin with this value.

For example:

    snd 1
    snd 2
    snd p
    rcv a
    rcv b
    rcv c
    rcv d

Both programs begin by sending three values to the other. Program 0 sends 1, 2, 0; program 1 sends 1, 2, 1. Then, each program receives a value (both 1) and stores it in a, receives another value (both 2) and stores it in b, and then each receives the program ID of the other program (program 0 receives 1; program 1 receives 0) and stores it in c. Each program now sees a different value in its own copy of register c.

Finally, both programs try to rcv a fourth time, but no data is waiting for either of them, and they reach a deadlock. When this happens, both programs terminate.

It should be noted that it would be equally valid for the programs to run at different speeds; for example, program 0 might have sent all three values and then stopped at the first rcv before program 1 executed even its first instruction.

Once both of your programs have terminated (regardless of what caused them to do so), how many times did program 1 send a value?

> Ok, almost all code in Proc can be used, we need to rewrite the op_snd abd op_rcv, add queue and an identifier. Also, we have another case to end the program: All the process are in blocked state (deadlock).

For example:

    set a 1
    add a 2
    mul a a
    mod a 5
    snd a
    set a 0
    rcv a
    jgz a -1
    set a 1
    jgz a -2

 - The first four instructions set a to 1, add 2 to it, square it, and then set it to itself modulo 5, resulting in a value of 4.
 
 - Then, a sound with frequency 4 (the value of a) is played.
 
 - After that, a is set to 0, causing the subsequent rcv and jgz instructions to both be skipped (rcv because a is 0, and jgz because a is not greater than 0).
 
- Finally, a is set to 1, causing the next jgz instruction to activate, jumping back two instructions to another jump, which jumps again to the rcv, which ultimately triggers the recover operation.

At the time the recover operation is executed, the frequency of the last sound played is 4.

In [425]:
from queue import Queue, Empty
from copy import copy

class Proc2(Proc):
    
    all_procs = set([])
    
    @classmethod
    def reset(cls):
        cls.all_procs = set([])

    def __init__(self, _id):
        super().__init__()
        self.blocked = False
        self.killed = False
        self._q = Queue()
        self._id = _id
        self.registers['p'] = _id
        Proc2.all_procs.add(self)

    def __str__(self):
        return '<P{} {} {} pc:{} [{}]>'.format(
            self._id,
            '✖' if self.blocked else '✓',
            '⛨' if self.killed else '≡',
            self.pc,
            self.dump(),
        )
            
        
    def op_snd(self, register_or_value):
        other_procs = [p for p in Proc2.all_procs if p._id != self._id]
        for p in other_procs:
            p._q.put(self.get_value(register_or_value))
        
    def op_rcv(self, register):
        self.blocked = True
        try:
            v = self._q.get_nowait()
            self.blocked = False
            self.registers[register] = v
            return True
        except Empty:
            return False
        
    def must_exit(self):
        self.killed = any([
            self.pc >= len(self.code),
            self.end_flag,
            all([p.blocked for p in Proc2.all_procs]),
            ])  
        return self.killed
    
    def load_program(self, lines, tron=False):
        self.stats = defaultdict(int)
        self.code = [_ for _ in lines]
        self.pc = 0
        self.blocked = False
        self.tron = tron
        
    def run(self):
        if self.must_exit():
            return 'end execution'
        buff = []    
        line = self.code[self.pc]
        if self.tron:
            buff.append('{:>5}{} |{}: {}|'.format(
                self._clock,
                '✖' if self.blocked else '✓',
                self.pc,
                self.code[self.pc],
                ))
            snapshot_before = copy(self.registers)
        result = self.exec(line)
        if result is None:
            self.pc += 1
            self._clock += 1
        if self.tron:
            if snapshot_before != self.registers:
                for r in self.registers:
                    if snapshot_before[r] != self.registers[r]: 
                        buff.append('{}[{} -> {}'.format(
                            r,
                            snapshot_before[r],
                            self.registers[r],
                            ))
        return ''.join(buff)


In [437]:
Proc2.reset()
p = Proc2(1)
assert p.pc == 0
p.load_program([
    'set a -1',
    'add a 1',
    'jgz 1 3',
    'set a 1',
    'set a 2',
    'set a 9',
    'mod a 7',
    'set a -1',
    'add a -2',
    'mul a -5'
], tron=True)
step = 0
while not p.killed:
    print('{:5d} {:22s} {:30s} {:25s}'.format(step, str(p), p.run(), str(p)))
    step += 1
assert p.registers.a == 15
assert p.stats['jgz'] == 1
assert p.stats['add'] == 2
assert p.stats['set'] == 3
assert p.stats['mod'] == 1
assert p.stats['mul'] == 1
print('ok')


    0 <P1 ✓ ≡ pc:0 [p=1]>        0✓ |0: set a -1|a[0 -> -1  <P1 ✓ ≡ pc:1 [a=-1:p=1]> 
    1 <P1 ✓ ≡ pc:1 [a=-1:p=1]>     1✓ |1: add a 1|a[-1 -> 0   <P1 ✓ ≡ pc:2 [a=0:p=1]>  
    2 <P1 ✓ ≡ pc:2 [a=0:p=1]>     2✓ |2: jgz 1 3|            <P1 ✓ ≡ pc:5 [a=0:p=1]>  
    3 <P1 ✓ ≡ pc:5 [a=0:p=1]>     3✓ |5: set a 9|a[0 -> 9    <P1 ✓ ≡ pc:6 [a=9:p=1]>  
    4 <P1 ✓ ≡ pc:6 [a=9:p=1]>     4✓ |6: mod a 7|a[9 -> 2    <P1 ✓ ≡ pc:7 [a=2:p=1]>  
    5 <P1 ✓ ≡ pc:7 [a=2:p=1]>     5✓ |7: set a -1|a[2 -> -1  <P1 ✓ ≡ pc:8 [a=-1:p=1]> 
    6 <P1 ✓ ≡ pc:8 [a=-1:p=1]>     6✓ |8: add a -2|a[-1 -> -3 <P1 ✓ ≡ pc:9 [a=-3:p=1]> 
    7 <P1 ✓ ≡ pc:9 [a=-3:p=1]>     7✓ |9: mul a -5|a[-3 -> 15 <P1 ✓ ≡ pc:10 [a=15:p=1]>
    8 <P1 ✓ ≡ pc:10 [a=15:p=1]> end execution                  <P1 ✓ ⛨ pc:10 [a=15:p=1]>
ok


> Lets go with the example:

In [434]:
import random

program = [
    'snd 1',
    'snd 2',
    'snd p',
    'rcv a',
    'rcv b',
    'rcv c',
    'rcv d',
    ]
Proc2.reset()
p0 = Proc2(0)
p1 = Proc2(1)
assert Proc2.all_procs == set([p0, p1])
p0.load_program(program, tron=True)
p1.load_program(program, tron=True)
step = 0
while not all([p.killed for p in Proc2.all_procs]):
    p = random.choice([p0, p1])
    print('{:5d} {:25s} {:^28s} {:25s}'.format(step, str(p), p.run(), str(p)))
    step += 1
assert p0.killed
assert p1.killed
print(p0.stats, p1.stats)
print('OK')

    0 <P1 ✓ ≡ pc:0 [p=1]>                0✓ |0: snd 1|       <P1 ✓ ≡ pc:1 [p=1]>      
    1 <P1 ✓ ≡ pc:1 [p=1]>                1✓ |1: snd 2|       <P1 ✓ ≡ pc:2 [p=1]>      
    2 <P0 ✓ ≡ pc:0 [p=0]>                0✓ |0: snd 1|       <P0 ✓ ≡ pc:1 [p=0]>      
    3 <P1 ✓ ≡ pc:2 [p=1]>                2✓ |2: snd p|       <P1 ✓ ≡ pc:3 [p=1]>      
    4 <P1 ✓ ≡ pc:3 [p=1]>            3✓ |3: rcv a|a[0 -> 1   <P1 ✓ ≡ pc:3 [a=1:p=1]>  
    5 <P0 ✓ ≡ pc:1 [p=0]>                1✓ |1: snd 2|       <P0 ✓ ≡ pc:2 [p=0]>      
    6 <P0 ✓ ≡ pc:2 [p=0]>                2✓ |2: snd p|       <P0 ✓ ≡ pc:3 [p=0]>      
    7 <P1 ✓ ≡ pc:3 [a=1:p=1]>        3✓ |3: rcv a|a[1 -> 2   <P1 ✓ ≡ pc:3 [a=2:p=1]>  
    8 <P1 ✓ ≡ pc:3 [a=2:p=1]>        3✓ |3: rcv a|a[2 -> 0   <P1 ✓ ≡ pc:3 [a=0:p=1]>  
    9 <P0 ✓ ≡ pc:3 [p=0]>            3✓ |3: rcv a|a[0 -> 1   <P0 ✓ ≡ pc:3 [a=1:p=0]>  
   10 <P0 ✓ ≡ pc:3 [a=1:p=0]>        3✓ |3: rcv a|a[1 -> 2   <P0 ✓ ≡ pc:3 [a=2:p=0]>  
   11 <P0 ✓ ≡ pc:3 [a=2:p=0]>        3✓ |3:

> Ok, let's go try it with the real code

In [456]:
with open('input.txt', 'r') as f:
    program = [_.strip() for _ in f.readlines() if _.strip()]

Proc2.reset()
p0 = Proc2(0)
p0.load_program(program, tron=True)
p1 = Proc2(1)
p1.load_program(program, tron=True)

print(p0, p1)

def trace(p):
    return 'Tri{:<38s} {:25s}'.format(p.run(), str(p))

step = 0
while not p1.blocked:
    print(trace(p1))
    step += 1

print(p0, p1)
print(p1.stats['snd'])

while not p0.blocked:
    print(trace(p0))
    step += 1
    
p1.run()


<P0 ✓ ≡ pc:0 [p=0]> <P1 ✓ ≡ pc:0 [p=1]>
Tri    0✓ |0: set i 31|i[0 -> 31          <P1 ✓ ≡ pc:1 [i=31:p=1]> 
Tri    1✓ |1: set a 1|a[0 -> 1            <P1 ✓ ≡ pc:2 [i=31:a=1:p=1]>
Tri    2✓ |2: mul p 17|p[1 -> 17          <P1 ✓ ≡ pc:3 [i=31:a=1:p=17]>
Tri    3✓ |3: jgz p p|                    <P1 ✓ ≡ pc:20 [i=31:a=1:p=17]>
Tri    4✓ |20: jgz a 3|                   <P1 ✓ ≡ pc:23 [i=31:a=1:p=17]>
Tri    5✓ |23: set f 0|                   <P1 ✓ ≡ pc:24 [i=31:a=1:p=17:f=0]>
Tri    6✓ |24: set i 126|i[31 -> 126      <P1 ✓ ≡ pc:25 [i=126:a=1:p=17:f=0]>
Tri    7✓ |25: rcv a|                     <P1 ✖ ≡ pc:25 [i=126:a=1:p=17:f=0]>
<P0 ✓ ≡ pc:0 [p=0]> <P1 ✖ ≡ pc:25 [i=126:a=1:p=17:f=0]>
0
Tri    0✓ |0: set i 31|i[0 -> 31          <P0 ✓ ≡ pc:1 [i=31:p=0]> 
Tri    1✓ |1: set a 1|a[0 -> 1            <P0 ✓ ≡ pc:2 [i=31:a=1:p=0]>
Tri    2✓ |2: mul p 17|                   <P0 ✓ ≡ pc:3 [i=31:a=1:p=0]>
Tri    3✓ |3: jgz p p|                    <P0 ✓ ≡ pc:4 [i=31:a=1:p=0]>
Tri    4✓ |4: mul a 2|a[1 -> 2 

In [370]:
with open('input.txt', 'r') as f:
    program = [_.strip() for _ in f.readlines() if _.strip()]
    
Proc2.reset()
p0 = Proc2(0)
p0.load_program(program, tron=True)
p1 = Proc2(1)
p1.load_program(program, tron=True)

def trace(p):
    return '{:<38s} {:25s}'.format(p.run(), str(p))
    
for i in range(122):
    print(i, traza(p0))

0     0✓ |0: set i 31|i[0 -> 31          Proc:0 ✓ ≡ pc:1 [i=31:p=0]
1     1✓ |1: set a 1|a[0 -> 1            Proc:0 ✓ ≡ pc:2 [i=31:a=1:p=0]
2     2✓ |2: mul p 17|                   Proc:0 ✓ ≡ pc:3 [i=31:a=1:p=0]
3     3✓ |3: jgz p p|                    Proc:0 ✓ ≡ pc:4 [i=31:a=1:p=0]
4     4✓ |4: mul a 2|a[1 -> 2            Proc:0 ✓ ≡ pc:5 [i=31:a=2:p=0]
5     5✓ |5: add i -1|i[31 -> 30         Proc:0 ✓ ≡ pc:6 [i=30:a=2:p=0]
6     6✓ |6: jgz i -2|                   Proc:0 ✓ ≡ pc:4 [i=30:a=2:p=0]
7     7✓ |4: mul a 2|a[2 -> 4            Proc:0 ✓ ≡ pc:5 [i=30:a=4:p=0]
8     8✓ |5: add i -1|i[30 -> 29         Proc:0 ✓ ≡ pc:6 [i=29:a=4:p=0]
9     9✓ |6: jgz i -2|                   Proc:0 ✓ ≡ pc:4 [i=29:a=4:p=0]
10    10✓ |4: mul a 2|a[4 -> 8            Proc:0 ✓ ≡ pc:5 [i=29:a=8:p=0]
11    11✓ |5: add i -1|i[29 -> 28         Proc:0 ✓ ≡ pc:6 [i=28:a=8:p=0]
12    12✓ |6: jgz i -2|                   Proc:0 ✓ ≡ pc:4 [i=28:a=8:p=0]
13    13✓ |4: mul a 2|a[8 -> 16           Proc:0 ✓ ≡ pc:5 [i=28:a

In [355]:
while not p0.blocked:
    traza(p0)

Proc:0 ✓ ≡ pc:0 [p=0]         0✓ |0: set i 31|i[0 -> 31 Proc:0 ✓ ≡ pc:1 [i=31:p=0]
Proc:0 ✓ ≡ pc:1 [i=31:p=0]     1✓ |1: set a 1|a[0 -> 1  Proc:0 ✓ ≡ pc:2 [i=31:a=1:p=0]
Proc:0 ✓ ≡ pc:2 [i=31:a=1:p=0]         2✓ |2: mul p 17|     Proc:0 ✓ ≡ pc:3 [i=31:a=1:p=0]
Proc:0 ✓ ≡ pc:3 [i=31:a=1:p=0]         3✓ |3: jgz p p|      Proc:0 ✓ ≡ pc:4 [i=31:a=1:p=0]
Proc:0 ✓ ≡ pc:4 [i=31:a=1:p=0]     4✓ |4: mul a 2|a[1 -> 2  Proc:0 ✓ ≡ pc:5 [i=31:a=2:p=0]
Proc:0 ✓ ≡ pc:5 [i=31:a=2:p=0]     5✓ |5: add i -1|i[31 -> 30 Proc:0 ✓ ≡ pc:6 [i=30:a=2:p=0]
Proc:0 ✓ ≡ pc:6 [i=30:a=2:p=0]         6✓ |6: jgz i -2|     Proc:0 ✓ ≡ pc:4 [i=30:a=2:p=0]
Proc:0 ✓ ≡ pc:4 [i=30:a=2:p=0]     7✓ |4: mul a 2|a[2 -> 4  Proc:0 ✓ ≡ pc:5 [i=30:a=4:p=0]
Proc:0 ✓ ≡ pc:5 [i=30:a=4:p=0]     8✓ |5: add i -1|i[30 -> 29 Proc:0 ✓ ≡ pc:6 [i=29:a=4:p=0]
Proc:0 ✓ ≡ pc:6 [i=29:a=4:p=0]         9✓ |6: jgz i -2|     Proc:0 ✓ ≡ pc:4 [i=29:a=4:p=0]
Proc:0 ✓ ≡ pc:4 [i=29:a=4:p=0]    10✓ |4: mul a 2|a[4 -> 8  Proc:0 ✓ ≡ pc:5 [i=29:a=8:p=0]
Proc:0 

In [356]:
while not p1.blocked:
    traza(p1)

Proc:1 ✓ ≡ pc:0 [p=1]         0✓ |0: set i 31|i[0 -> 31 Proc:1 ✓ ≡ pc:1 [i=31:p=1]
Proc:1 ✓ ≡ pc:1 [i=31:p=1]     1✓ |1: set a 1|a[0 -> 1  Proc:1 ✓ ≡ pc:2 [i=31:a=1:p=1]
Proc:1 ✓ ≡ pc:2 [i=31:a=1:p=1]     2✓ |2: mul p 17|p[1 -> 17 Proc:1 ✓ ≡ pc:3 [i=31:a=1:p=17]
Proc:1 ✓ ≡ pc:3 [i=31:a=1:p=17]         3✓ |3: jgz p p|      Proc:1 ✓ ≡ pc:20 [i=31:a=1:p=17]
Proc:1 ✓ ≡ pc:20 [i=31:a=1:p=17]         4✓ |20: jgz a 3|     Proc:1 ✓ ≡ pc:23 [i=31:a=1:p=17]
Proc:1 ✓ ≡ pc:23 [i=31:a=1:p=17]         5✓ |23: set f 0|     Proc:1 ✓ ≡ pc:24 [i=31:a=1:p=17:f=0]
Proc:1 ✓ ≡ pc:24 [i=31:a=1:p=17:f=0]     6✓ |24: set i 126|i[31 -> 126 Proc:1 ✓ ≡ pc:25 [i=126:a=1:p=17:f=0]
Proc:1 ✓ ≡ pc:25 [i=126:a=1:p=17:f=0]     7✓ |25: rcv a|a[1 -> 7656 Proc:1 ✓ ≡ pc:25 [i=126:a=7656:p=17:f=0]
Proc:1 ✓ ≡ pc:25 [i=126:a=7656:p=17:f=0]     7✓ |25: rcv a|a[7656 -> 8238 Proc:1 ✓ ≡ pc:25 [i=126:a=8238:p=17:f=0]
Proc:1 ✓ ≡ pc:25 [i=126:a=8238:p=17:f=0]     7✓ |25: rcv a|a[8238 -> 6004 Proc:1 ✓ ≡ pc:25 [i=126:a=6004:p=17:f=0]
P

In [358]:
p0.blocked, p1.blocked

(True, True)

In [150]:
p0.dump(), p1.dump(), p0._q.qsize(), p1._q.qsize()

('a=1:p=0:b=2:c=1', 'a=1:p=1:b=2:c=0', 0, 0)

In [153]:
p0.code[p0.pc]

'rcv d'

In [154]:
p0.run()

KeyboardInterrupt: 